# Creating component from Airflow operators
Airflow is a popular orchestrator similar to Pipelines.
Airflow's "operators" are pretty similar to "components". Airflow Operators lack some of the features of Kubeflow Piplines components like well-defined component signature (inpus/outputs) and system-managed data passing between components.

Being able to re-use Airflow operators in Kubeflow Pipelines makes transitioning to Pipelines easier.

To convert the Airflow operator to Pipelines component use the [create_component_from_airflow_op](https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.components.html#kfp.components.create_component_from_airflow_op) function to get component function that you can use in a pipleine.

## create_component_from_airflow_op
The `create_component_from_airflow_op` function creates component function from an Airflow operator class.
The inputs of the component are the same as the operator constructor parameters.
By default the component has the following outputs: "Result", "Variables" and "XComs". "Variables" and "XComs" are serialized JSON maps of all variables and xcoms produced by the operator during the execution.
Use the variable_output_names and xcom_output_names parameters to output individual variables/xcoms as separate outputs.

Args:
* op_class: Reference to the Airflow operator class (e.g. EmailOperator or BashOperator) to convert to componenent.
* base_image: Optional. The container image to use for the component. Default is apache/airflow. The container image must have the same python version as the environment used to run create_component_from_airflow_op. The image should have python 3.5+ with airflow package installed.
* variable_output_names: Optional. A list of Airflow "variables" produced by the operator that should be returned as separate outputs.
* xcom_output_names: Optional. A list of Airflow "XComs" produced by the operator that should be returned as separate outputs.
* modules_to_capture: Optional. A list of names of additional modules that the operator depends on. By default only the module containing the operator class is captured. If the operator class uses the code from another module, the name of that module can be specified in this list.

In [ ]:
# Install Kubeflow Pipelines SDK
!PIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install 'kfp>=0.1.31.2' --quiet
!PIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install apache-airflow --quiet

In [ ]:
import kfp
from kfp.components import create_component_from_airflow_op

In [ ]:
from airflow.operators.bash_operator import BashOperator

# Creating component from Airflow operator
bash_op = create_component_from_airflow_op(BashOperator)

In [ ]:
def my_pipeline():
    bash_task = bash_op(
        bash_command='echo "Hello from $SHELL"',
        xcom_push=True,
    )

In [ ]:
kfp.Client().create_run_from_pipeline_func(my_pipeline, arguments={})